In [1]:
from atopt.core.constrainedcsp import constrained_CSP
from atopt.utilities import *

from datetime import datetime
from pathlib import Path
from typing import Union
from docplex.cp.model import *
import docplex.cp.utils_visu as visu
import matplotlib.pyplot as plt
from pylab import rcParams

In [2]:
DATAFILE = "C:/Users/aznavouridis.k/My Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"
SAVELOC = "D:/.temp/.dev/.aztool/atopt/sols"
ROUTE = '910'

d = DataProvider(filepath=DATAFILE, route=ROUTE, adjust_for_traffic=True)

model = CSPModel(d)
model.build_model()

cp_model, model_info = constrained_CSP(model=model, nduties=20)
cpsol = cp_model.solve(TimeLimit=60)

In [3]:
print(cpsol.get_solve_status())

Feasible


In [4]:
print(cpsol.print_solution())

-------------------------------------------------------------------------------
Model constraints: 3580, variables: integer: 1160, interval: 1200, sequence: 20
Solve status: Feasible
Search status: SearchStopped, stop cause: SearchStoppedByLimit
Solve time: 60.79 sec
-------------------------------------------------------------------------------
Objective values: (8,), bounds: (5,), gaps: (0.375,)
EDT_00_00=40
EDT_00_01=40
EDT_00_02=40
EDT_00_03=40
EDT_00_04=40
EDT_00_05=40
EDT_00_06=40
EDT_00_07=40
EDT_00_08=40
EDT_00_09=40
EDT_00_10=40
EDT_00_11=40
EDT_00_12=40
EDT_00_13=40
EDT_00_14=40
EDT_00_15=40
EDT_00_16=40
EDT_00_17=40
EDT_00_18=40
EDT_00_19=40
EDT_01_00=40
EDT_01_01=40
EDT_01_02=40
EDT_01_03=40
EDT_01_04=40
EDT_01_05=40
EDT_01_06=40
EDT_01_07=40
EDT_01_08=40
EDT_01_09=40
EDT_01_10=40
EDT_01_11=40
EDT_01_12=40
EDT_01_13=40
EDT_01_14=40
EDT_01_15=40
EDT_01_16=40
EDT_01_17=40
EDT_01_18=40
EDT_01_19=40
EDT_02_00=40
EDT_02_01=40
EDT_02_02=40
EDT_02_03=40
EDT_02_04=40
EDT_02_05=40
E

In [ ]:
rcParams['figure.figsize'] = 10, 4

model = model_info.get('model')
ntrips = model_info.get('ntrips')
nduties = model_info.get('nduties')
duties = model_info.get('duties')
trip2duty = model_info.get('trip2duty')
breaks = model_info.get('breaks')
min_start = model_info.get('min_start')
max_end = model_info.get('max_end')

bounds = sol.get_objective_bounds()
status = sol.get_solve_status()

save_loc = Path(save_folder)

date_str = datetime.now().strftime('[%Y-%m-%d_%H-%M-%S]')
out = save_loc.joinpath(f"{date_str}_{status}_{bounds[0]}.txt")
sol_log = save_loc.joinpath(f"{date_str}_{status}_{bounds[0]}_SOLUTION.txt")

sol.print_solution()

with open(sol_log, 'w') as sol_log_file:
    driving_times = []
    for d in range(nduties):
        if sol[duties[d]]:
            print(f"\n> Duty {d} : {sol[duties[d]]}")
            sol_log_file.write(f"\n> Duty {d} : {sol[duties[d]]}")
            # print(cdt[d])
            _tdt = 0
            _ntrips = 0
            for t in range(ntrips):
                if sol[trip2duty[(t, d)]]:
                    print(f"  - Trip {t} : {sol[trip2duty[(t, d)]]}")
                    sol_log_file.write(
                        f"\n  - Trip {t} : {sol[trip2duty[(t, d)]]}")
                    _tdt += model.durations[t]
                    _ntrips += 1
            print(f"\n  > Driving Time: {_tdt}, Trips: {_ntrips}")
            sol_log_file.write(
                f"\n  > Driving Time: {_tdt}, Trips: {_ntrips}\n")
            driving_times.append(_tdt)
        else:
            driving_times.append(0)

        # print(cdt[d])
sol.write(str(out))

visu.timeline(
    f"{date_str}_{status}_{bounds[0]}", origin=min_start, horizon=max_end)

for d in range(nduties):
    if sol[duties[d]]:
        visu.panel()
        visu.sequence(name=f"{duties[d].get_name()} ({driving_times[d]})",
                        intervals=[(sol.get_var_solution(trip2duty[(t, d)]), d, str(t)) for t in range(ntrips) if sol[trip2duty[(t, d)]]])
        visu.interval(sol.get_var_solution(breaks[d]), 'red', 'B')

visu.show()